# Descrição

Não é necessário utilizar a GPU, a execução é rápida.

Este notebook baixa as 269 emendas do tipo CCJ anotadas pelo consultor. Utiliza-se:

- a versão supervisionada do BERTopic, que possui um classificador, que é alimentado pela representação do texto através de embeddings;

- o texto **completo** da emenda, sem nenhum tipo de pré processamento ou segmentação

- os tópicos anotados pelo consultor são passados como 'target' para o modelo


# Instalações

In [1]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=a0fa1869abe29cfc09c64103e3a9fc73a5a70676532b4eec07f63cb5b388ad4d
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


In [2]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# ⚠ ⚠ ⚠

A célula referente à instalação que der erro deve ser executada novamente (executá-la 2 vezes):

In [3]:
!pip install --upgrade joblib==1.1.0

!pip install hdbscan

!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.0/307.0 KB 8.2 MB/s eta 0:00:00
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.1
    Uninstalling joblib-1.1.1:
      Successfully uninstalled joblib-1.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-learn 1.2.2 requires joblib>=1.1.1, but you have joblib 1.1.0 which is incompatible.
imbalanced-learn 0.10.1 requires joblib>=1.1.1, but you have joblib 1.1.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 23.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) 

In [5]:
import pandas as pd
import numpy as np
import gdown
import seaborn as sns
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.dimensionality import BaseDimensionalityReduction
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

# Baixando a base de dados com as CCJ's e seus respectivos tópicos:

In [6]:
id = "1-60Vyv8j8uSm5_I4hi05aV6s_LFny86Y"
output = "base_dados_CCJs_e_tópicos.xlsx"
gdown.download(id=id, output=output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1-60Vyv8j8uSm5_I4hi05aV6s_LFny86Y
To: /content/base_dados_CCJs_e_tópicos.xlsx
100%|██████████| 293k/293k [00:00<00:00, 70.4MB/s]


'base_dados_CCJs_e_tópicos.xlsx'

In [7]:
df = pd.read_excel("base_dados_CCJs_e_tópicos.xlsx")
df.head(5)
# link desta base:
# https://docs.google.com/spreadsheets/d/1-60Vyv8j8uSm5_I4hi05aV6s_LFny86Y/edit?usp=sharing&ouid=112604482026911414314&rtpof=true&sd=true

,nome,texto,todos_os_tópicos
0,CCJ_1,...,custeio
1,CCJ_2,...,pensão mínima
2,CCJ_3,...,Regime Geral de Previdência Social de transição
3,CCJ_4,...,estados e municípios
4,CCJ_5,...,militares


In [8]:
len(df)

269

# Fazendo o encoding dos nossos labels/tópicos

Note que o pipeline do BERTopic de modo supervisionado utiliza um classificador (regressão logística) para "agrupar" os documentos em cada classe/tópico. 

Referência: https://maartengr.github.io/BERTopic/getting_started/supervised/supervised.html

Assim, o BERTopic deve receber os 'tópicos' já anotados através de uma representação de classes abstraída poruma variável numérica. Por exemplo, o tópico "Abono Permanente" representa a classe 0, o tópico "Abono Salarial" representa a classe 1, etc. Portanto, usamos um LabelEncoder para transformar os tópicos anotados em uma representação numérica:

In [9]:
label_encoder = LabelEncoder()
 
# transformando cada tópico da nossa base em uma representação numérica:
label = label_encoder.fit_transform(df['todos_os_tópicos'])

# inserindo a representação numérica de cada classe/tópico em nosso dataframe:
df['encoded_label'] = label  

df

,nome,texto,todos_os_tópicos,encoded_label
0,CCJ_1,...,custeio,14
1,CCJ_2,...,pensão mínima,21
2,CCJ_3,...,Regime Geral de Previdência Social de transição,10
3,CCJ_4,...,estados e municípios,16
4,CCJ_5,...,militares,18
...,...,...,...,...
264,CCJ_265,...,pensão,20
265,CCJ_266,...,pensão mínima,21
266,CCJ_267,...,pensão,20
267,CCJ_268,...,pensão,20


# Fazendo o Fit da BERTopic supervisionada:

Note que estamos passando o texto **completo** da CCJ ao modelo, sem utilizar o Ulysses Segmenter para fazer a limpeza do texto e remover a justificativa. Assim fica mais comparável aos experimentos do Flávio, que foram feitos com a LookForSimilar, com o texto inteiro. 

Num segundo experimento, podemos "limpar" o texto e remover a justificativa com o segmentador do Felipe.

Devemos passar a representação numérica (encoded labels) para a BERTopic supervisionada. Utilizamos o modelo multilingual (paraphrase-multilingual-MiniLM-L12-v2)

In [10]:

docs = df['texto']
y = df['encoded_label']

# Skip over dimensionality reduction, replace cluster model with classifier,
# and reduce frequent words while we are at it.
empty_dimensionality_model = BaseDimensionalityReduction()
clf = LogisticRegression()
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

# Create a fully supervised BERTopic instance
topic_model= BERTopic(
        umap_model=empty_dimensionality_model,
        hdbscan_model=clf,
        ctfidf_model=ctfidf_model,
        language="multilingual"
)
topics, probs = topic_model.fit_transform(docs, y=y)

# Resultado

No dataframe abaixo, "Name" são os tópicos gerados pela BERTopic supervisionada, e "Class" são os tópicos anotados pelo consultor. Nota-se que um pré processamento melhor do texto melhoraria nossos resultados:

In [11]:
# Mapeando os tópicos anotados (strings) em relação aos label encoders númericos obtidos:
mappings = topic_model.topic_mapper_.get_mappings()
mappings = {value: df["todos_os_tópicos"][key] for key, value in mappings.items()}

# Colocando uma coluna em nosso dataframe com o tópico anotado:
df = topic_model.get_topic_info()
df["Class"] = df.Topic.map(mappings)
df

,Topic,Count,Name,Class
0,0,28,0_alíquotas_confiscatório_progressivas_tributo,trabalhadores rurais
1,1,25,1_serviço_anos_transição_público,outros
2,2,20,2_dependente_23_pensão_morte,pensão
3,3,19,3_média_26_cálculo_acréscimo,pensão
4,4,18,4_extinção_vantagens_34_ente,pensão
5,5,15,5_suez_batalhão_paz_ex,Regime Geral de Previdência Social / Regime Pr...
6,6,14,6_203_bpc_critério_per,Regime Próprio de Previdência Social
7,7,13,7_formal_salário_morte_fonte,Oficial de Justiça
8,8,13,8_pedágio_20_iv_transição,magistério
9,9,11,9_conversão_especial_agentes_exposição,estados e municípios
